[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hpcgarage/pyboot-g2s3/blob/master/supplemental/tricount.ipynb)

# Exercises: Triangle counting via sparse linear algebra

This problem is about counting triangles in a graph, which has applications in social network analysis.

## Background: Counting triangles in a social network

A social network may be modeled as an undirected graph, like the one shown below.

![An example of an undirected graph](./graph.png)

The _nodes_ (or _vertices_) of this graph, shown as numbered circles, might represent people, and the _edges_ (or links connecting them) might represent who is friends with whom. In this case, person 0 is friends with all the "odd birds" of this network, persons 1, 3, and 5, but has no direct connection to persons 2 and 4.

One type of analysis one might perform on such a graph is _counting triangles_, that is, the number of relationships of the form $a$ knows $b$, $b$ knows $c$, and $c$ knows $a$. In the graph shown above, there are two such triangles: (0, 1, 3) and (0, 3, 5).

## Counting triangles using a "standard" graph package

A commonly used Python-based library for graph computations is [NetworkX](https://networkx.github.io/). Let's use it to build the above graph and count the triangles.

In [ ]:
import networkx
%matplotlib inline

V = list(range(6))
E = [(0, 1), (0, 3), (0, 5), (1, 3), (5, 3), (3, 2), (3, 4)]
G = networkx.Graph()
G.add_nodes_from(V)
G.add_edges_from(E)

def count_triangles_nx(G, ret_tricounts=False):
    """
    Given a NetworkX graph, `G`, this function returns the total number of
    triangles it contains. If the argument `ret_C=True`, then it also returns
    a dictionary where each (key, value) pair is the vertex and the number of
    triangles that include that vertex.
    """
    from networkx.algorithms.cluster import triangles
    tricounts = triangles(G)
    n_tri = sum(tricounts.values()) // 3
    if ret_tricounts:
        return n_tri, tricounts
    return n_tri

networkx.draw(G, with_labels=True)
ntri_G, tricounts_G = count_triangles_nx(G, ret_tricounts=True)
print("This graph has {} triangles:".format(ntri_G))
print("Node-triangle occurrence counts:", tricounts_G)

## A matrix-based method

**Adjacency matrix.** Let $A$ be the _adjacency matrix_ representation of the graph, defined as follows. The entries of $A$ are either 0 or 1; and $a_{i,j}$ equals 1 if and only if there is an edge connecting nodes $i$ and $j$. For instance, for the graph shown above,

$$
A = \begin{bmatrix}
        0 & 1 & 0 & 1 & 0 & 1 \\
        1 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 \\
        1 & 1 & 1 & 0 & 1 & 1 \\
        0 & 0 & 0 & 1 & 0 & 0 \\
        1 & 0 & 0 & 1 & 0 & 0
    \end{bmatrix}.
$$

Observe that the relationships are symmetric. For instance, 0 and 1 are mutually connected; therefore, both $a_{0,1}$ and $a_{1, 0}$ equal 1, and in general, $A = A^T$.

Given this representation of the graph, here is one way to count triangles using linear algebra.

First, let $A \cdot B$ denote matrix multiplication. That is, $C = A \cdot B$ means $c_{i,j} = \sum_k a_{i,k} b_{k, j}$.

Next, let $A \odot B$ denote _elementwise_ multiplication. That is, $E = A \odot B$ means $e_{i, j} = a_{i, j} b_{i, j}$.

Then, here is a two-step method to compute the number of triangles, which we'll denote as $t(A)$:

$$
\begin{eqnarray}
       C & = & (A \cdot A) \odot A \\
    t(A) & = & \frac{1}{6} \sum_{i, j} c_{i,j}.
\end{eqnarray}
$$

The first step computes a "count" matrix $C$. Each element, $c_{i,j}$, counts the number of triangles in which both $i$ and $j$ appear. For the example shown above, it turns out that $c_{0, 1} = c_{1,0} = 1$ since there is only one triangle that uses the edge $(0, 1)$, whereas $c_{0, 3} = c_{3, 0} = 2$ because the edge $(0, 3)$ appears in two triangles.

The second step sums all the elements of $C$. However, the sum alone will overcount the number of unique triangles by a factor of six (6), hence the additional factor of $\frac{1}{6}$. (Why?)

> Instead of summing all the entries of $A$, one can exploit symmetry and consider just the upper- or lower-triangle, but more on that later!

**Exercise 0** (3 points). Implement a function, **`count_triangles(A)`**, that implements the above formula. That is, given a symmetric Numpy array `A` representing the adjacency matrix of a graph, this function will return the number of triangles. Assume the input matrix `A` is a (dense!) Numpy array.

> There is an assertion on the first line that checks this latter condition.

Your implementation should return a value whose type is either a Python or Numpy integer.

In [ ]:
import numpy

def count_triangles(A, ret_C=False):
    assert (type(A) is numpy.ndarray) and (A.ndim == 2) and (A.shape[0] == A.shape[1])
    ###
    ### YOUR CODE HERE
    ###
    
A = networkx.convert_matrix.to_numpy_array(G, dtype=int)
print(count_triangles(A, ret_C=True))

In [ ]:
# Test cell: `count_triangles_test`

def is_int(x):
    return isinstance(x, numpy.integer) or isinstance(x, int)

ntri_A = count_triangles(A)
assert is_int(ntri_A), f"You returned a value of type `{type(ntri_A)}` instead of a Python or Numpy integer."
assert ntri_A == 2, f"The graph only has 2 triangles, not {ntri_A}"

print("\n(Passed part 1.)")

In [ ]:
# Test cell: `count_triangles_test2`
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

def check_count_triangles_large(n, den=1e-2):
    U_large = numpy.triu(numpy.random.rand(n, n) <= den).astype(int)
    numpy.fill_diagonal(U_large, 0)
    A_large = U_large + U_large.T
    return count_triangles(A_large)

n, den, k_max, mu, sd = 500, 1e-2, 25, 21, 5
nts = numpy.zeros(k_max, dtype=int)
for k in range(k_max):
    nts[k] = check_count_triangles_large(n, den)
    print(k, nts[k], numpy.sum(nts[:k+1])/(k+1))
sns.distplot(nts)
plt.xlabel("Number of triangles")
plt.ylabel("Count")
plt.title("{} trials: {} nodes, uniform random connections, {}% fill".format(k_max, n, den*100))

assert (mu-sd) <= numpy.mean(nts) <= (mu+sd), "mean={}".format(numpy.mean(nts))

## Performance bake-off: An actor network dataset

Let's apply the triangle counts to a real "social" network, namely, the graph of actors who have appeared in the same movie. The dataset in this problem uses data collected on a crawl of the [Top 250 movies on the Internet Movie Database](https://github.com/napsternxg/IMDB-Graph/tree/master/Data/tutorial/tutorial) (circa 2012).

Let's start by loading this data, which is contained in a pair of JSON files.

> These data should be available in the repository containing this notebook. However, an additional copy is available at the following URL, but you may need to adapt the code below to use it. https://cse6040.gatech.edu/datasets/imdb.zip

In [ ]:
import json

def fn(fn_base, dirname='./imdb/'):
    return "{}{}".format(dirname, fn_base)

def load_json(basefile):
    filename = fn(basefile)
    json_file = open(filename, encoding="utf8")
    json_str = json_file.read()
    json_data = json.loads(json_str)
    return json_data

movies_json = load_json("imdb.json")
casts_json = load_json("imdb_cast.json")

**About the data.** There are two parts to the data.

The first is `movies_json`, which is a JSON formatted collection of movie titles and IDs. It is a list and here are the first few entries:

In [ ]:
print("=== First five entries of `movies_json` ===\n")
for k, m in enumerate(movies_json[:5]):
    print("[{}] {}".format(k, m))
print("...")

The second part is `casts_json`, which is a JSON formatted collection of movies with information about who appeared in the movie. It is also a list and here is the very first element.

Observe that it is a dictionary with information about a single movie, including the movie's title, it's IMDB URL, and list of actors (cast members).

In [ ]:
print("=== First entry of `casts_json` ===\n")
print(json.dumps(casts_json[0], indent=3))

**Exercise 1** (2 points). Implement a function that, given the `casts_json` list, returns a dictionary that maps actor links to actor names.

In the example above, the first actor listed for "12 Angry Men" is `"Martin Balsam"`, whose link is `"/name/nm0000842/"`. Therefore, the dictionary that your function returns should include the key-value pair, `"/name/nm0000842/" : "Martin Balsam"`.

> _Hint._ Pay careful attention to the structure of the output above.

In [ ]:
def gather_actors(casts):
    actors = dict() # Use to store (actor link) : (actor name) pairs
    for movie in casts:
        assert "cast" in movie
        ###
        ### YOUR CODE HERE
        ###
    return actors

actors = gather_actors(casts_json)
print("Found {} unique actors.\n".format(len(actors)))

assert "/name/nm0000842/" in actors
print("'{}' -> '{}'".format("/name/nm0000842/", actors["/name/nm0000842/"]))
assert actors["/name/nm0000842/"] == "Martin Balsam"

In [ ]:
# Test cell: `gather_actors_test`

assert ("/name/nm0872820/" in actors) and (actors["/name/nm0872820/"] == "Amedeo Trilli")
assert ("/name/nm0279786/" in actors) and (actors["/name/nm0279786/"] == "Shug Fisher")
assert ("/name/nm0802831/" in actors) and (actors["/name/nm0802831/"] == "Tony Sirico")
assert ("/name/nm0924692/" in actors) and (actors["/name/nm0924692/"] == "Dean White")
assert ("/name/nm0248074/" in actors) and (actors["/name/nm0248074/"] == "Jake Eberle")
assert ("/name/nm1067542/" in actors) and (actors["/name/nm1067542/"] == "Grace Keller")
assert ("/name/nm0903694/" in actors) and (actors["/name/nm0903694/"] == "Carl Voss")
assert ("/name/nm1504897/" in actors) and (actors["/name/nm1504897/"] == "Radka Kucharova")
assert ("/name/nm0644905/" in actors) and (actors["/name/nm0644905/"] == "Tae-kyung Oh")
assert ("/name/nm0727037/" in actors) and (actors["/name/nm0727037/"] == "Gary Riley")
assert ("/name/nm2006011/" in actors) and (actors["/name/nm2006011/"] == "Glenn Stanton")
assert ("/name/nm0193389/" in actors) and (actors["/name/nm0193389/"] == "John Curtis")
assert ("/name/nm0829189/" in actors) and (actors["/name/nm0829189/"] == "Avril Stewart")
assert ("/name/nm1211469/" in actors) and (actors["/name/nm1211469/"] == "Karine Asure")
assert ("/name/nm0598388/" in actors) and (actors["/name/nm0598388/"] == "Jacques Monod")
assert ("/name/nm1663820/" in actors) and (actors["/name/nm1663820/"] == "Michael Garnet Stewart")
assert ("/name/nm0009388/" in actors) and (actors["/name/nm0009388/"] == "Khosrow Abrishami")
assert ("/name/nm0020513/" in actors) and (actors["/name/nm0020513/"] == "Fletcher Allen")
assert ("/name/nm0615419/" in actors) and (actors["/name/nm0615419/"] == "John Murtagh")
assert ("/name/nm0120165/" in actors) and (actors["/name/nm0120165/"] == "Keith S. Bullock")
assert ("/name/nm0448560/" in actors) and (actors["/name/nm0448560/"] == "Colin Kenny")
assert ("/name/nm0882139/" in actors) and (actors["/name/nm0882139/"] == "David Ursin")
assert ("/name/nm1597244/" in actors) and (actors["/name/nm1597244/"] == "Carol Meirelles")
assert ("/name/nm0316079/" in actors) and (actors["/name/nm0316079/"] == "Paul Giamatti")
assert ("/name/nm3546231/" in actors) and (actors["/name/nm3546231/"] == "Leonard B. John")

print("\n(Passed!)")

**Using NetworkX to count the triangles.** Let's define the actor network graph as follows. Each actor is a node; an edge exists between two actors if they appeared in the same movie.

Convince yourself that the following implementation computes the triangles correctly, assuming the NetworkX implementation is correct.

In [ ]:
def build_actor_network(casts):
    from itertools import combinations
    actors = gather_actors(casts)
    G = networkx.Graph()
    G.add_nodes_from(actors.keys())
    for movie in casts:
        for ai, aj in combinations(movie["cast"], 2):
            G.add_edge(ai['link'][0], aj['link'][0])
    return G

G_casts = build_actor_network(casts_json)
ntri_nx, tricounts_nx = count_triangles_nx(G_casts, ret_tricounts=True)
print("Found ~ {:.1f} million triangles in all.".format(ntri_nx*1e-6))

In [ ]:
t_nx = %timeit -o count_triangles_nx(G_casts) # Uncomment for timing info

**Exercise 2** (3 points). Try a matrix-based approach to counting triangles. Compare its speed against the NetworkX-based method above. As output, produce a dictionary named **`tricounts`**, where **`tricounts[link]`** stores the triangle count for the actor whose link is given by `link`.

> _Remark._ The matrix-based example we gave above stores the graph as a _dense_ matrix. However, the graph itself is actually quite sparse. Therefore, to get a scalable implementation, it will be critical to exploit the sparsity of the actor network. That is, observe that there are nearly $n=13,\!000$ actors in this dataset, so any method that scales like $\mathcal{O}(n^2)$ (or worse) is not likely to finish in a reasonable amount of time.

In [ ]:
###
### YOUR CODE HERE
###

In [ ]:
# Test cell

for actor in tricounts_nx:
    assert tricounts_nx[actor] == tricounts[actor], \
           "For actor '{}', NetworkX saw {} triangles whereas you saw {}.".format(actors[actor]
                                                                                  , tricounts_nx[actor]
                                                                                  , tricounts[actor])
print("\nPassed!")

**Fin!** That's the end of this problem.